# MATCHING MARKETS

Some theorems (courtesy of and using definitions from Harvard CS134):

* <strong>Thm0 (Existence of a Perfect Matching)</strong> :
    * A bipartite network $G$
        * (a network whose nodes are separable into two subsets $U$ and $V$
        * such that the neighborhood $N(u)$ of any node $u \in U$
        * does not include any other node $u' \in U$,
        * and similarly for $V$)
    * with $|U| = |V|$
    * has a perfect matching
    * iff it does not contain a set of constricted nodes
        * (a subset of nodes $U_C \subseteq U$
        * such that the neighborhood $N(U_C) = \bigcup_{u \in U_C} N(u)$ of $U_R$
        * is smaller than the number of nodes $|U_C|$---
        * or similarly for $V$).


* <strong>Thm1a (Existence of a Set of Socially-Optimal Market-Clearing Prices)</strong> :
    * For any set of buyer valuations (nonnegative real numbers) for each seller,
    * there exists a set of market-clearing seller prices (nonnegative real numbers)
    * that produces a socially-optimal outcome, i.e.
        * the induced perfect matching in the resulting preferred-seller network
            * (a bipartite subnetwork of the main network $G$
            * such that each buyer is linked to at most one seller --
            * either a single seller who maximizes the payoff to that buyer, with ties broken arbitrarily
            * or, if no seller can provide a nonnegative payoff, then no seller at all)
        * maximizes the sum of payoffs to all buyers and sellers.


* <strong>Thm1b (Social Optimality of Any Set of Market-Clearing Prices)</strong> :
    * For any set of market-clearing prices,
    * any perfect matching in the resulting preferred-seller network
    * is socially-optimal.